In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib

try:
   
    test_data = pd.read_csv('task1/test1.csv', parse_dates=['date'], dayfirst=True)

    print("Original Test Data:")
    print(test_data.head())

   
    test_data['date'] = test_data['date'].dt.strftime('%d-%m-%Y') 
    test_data['date'] = test_data['date'].str.strip() 


    if 'date' not in test_data.columns or test_data['date'].isnull().all():
        raise ValueError("Test data is invalid or contains no usable dates.")

   
    test_data = test_data.dropna(subset=['date']).reset_index(drop=True)
    print("\nCleaned Test Data:")
    print(test_data.head())

   
    if not test_data.empty:
        min_date = test_data['date'].min()
        test_data['days'] = (pd.to_datetime(test_data['date'], format='%d-%m-%Y') - pd.to_datetime(min_date, format='%d-%m-%Y')).dt.days
    else:
        raise ValueError("No valid test data is available for predictions.")

    print("\nTest Data with Days Column:")
    print(test_data.head())


    scaler = joblib.load('scaler.pkl')
    scaled_test_values = scaler.transform(test_data['days'].values.reshape(-1, 1))
    print("\nScaled Test Values:")
    print(scaled_test_values[:5])

   
    X_test = np.reshape(scaled_test_values, (scaled_test_values.shape[0], 1, 1))
    print("\nLSTM Input Shape:", X_test.shape)

    
    model = load_model('stock_price_model.h5')

    
    predictions = model.predict(X_test)
    print("\nScaled Predictions:")
    print(predictions[:5])

    
    predicted_prices = scaler.inverse_transform(predictions)
    print("\nInverse Transformed Predictions:")
    print(predicted_prices[:5])

   
    test_data['price'] = test_data['price'].fillna(pd.Series(predicted_prices.flatten()))

    print("\nTest Data with Predicted Prices:")
    print(test_data.head())

 
    test_data[['date', 'price']].to_csv('PREDICTION_RESULT.CSV', index=False)
    print("Predictions saved to 'PREDICTION_RESULT.CSV'.")

except Exception as e:
    print(f"An error occurred: {e}")



Original Test Data:
        date  price
0 2017-10-01    NaN
1 2017-10-02    NaN
2 2017-10-03    NaN
3 2017-10-04    NaN
4 2017-10-05    NaN

Cleaned Test Data:
         date  price
0  01-10-2017    NaN
1  02-10-2017    NaN
2  03-10-2017    NaN
3  04-10-2017    NaN
4  05-10-2017    NaN

Test Data with Days Column:
         date  price  days
0  01-10-2017    NaN     0
1  02-10-2017    NaN     1
2  03-10-2017    NaN     2
3  04-10-2017    NaN     3
4  05-10-2017    NaN     4

Scaled Test Values:
[[-0.08695652]
 [-0.06521739]
 [-0.04347826]
 [-0.02173913]
 [ 0.        ]]

LSTM Input Shape: (92, 1, 1)
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step

Scaled Predictions:
[[0.18452081]
 [0.19266832]
 [0.20085937]
 [0.20909365]
 [0.21737085]]

Inverse Transformed Predictions:
[[12.487958 ]
 [12.862743 ]
 [13.2395315]
 [13.618307 ]
 [13.999059 ]]

Test Data with Predicted Prices:
         date      price  days
0  01-10-2017  12.487958     0
1  02-10-2017  12.862743     1
2  03-10-2017  13.239532     2
3 